In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
from tqdm import tqdm
import os
from chinese_calendar import is_holiday, is_workday
import datetime
import geopandas
import networkx as nx  
from shapely.geometry import Point, LineString
from geodatasets import get_path 
import folium
from folium import plugins
from branca.colormap import LinearColormap
#分析目标地铁站在工作日早晚高峰时段的客流来源和去向。
"""
    输出结果：

返回两个列表：morningRushCount（早高峰各站点流入目标站的客流量）、eveningRushCount（晚高峰目标站流出到各站点的客流量）。
后续分析：

通过排序，找出早晚高峰客流量最多的前10个站点（即通勤来源和去向Top10）。
可以进一步可视化这些通勤流向，比如在地图上用箭头展示。
    """


In [2]:
stationInfo = pd.read_csv('../Data/stationinfo.csv')
dict_staionName = dict(zip(stationInfo['stationID'], stationInfo['name']))
dict_stationLoc = dict(zip(stationInfo['stationID'], zip(stationInfo['lon'],stationInfo['lat'])))
stationID_List = list(stationInfo['stationID'])

In [ ]:
def RushHourConnection(targetStationID, stationID_List): # Flow Mon-Fri

    morningRushCount = [0] * len(stationID_List) # Morning Orgin
    eveningRushCount = [0] * len(stationID_List) # evening destination
    currDate = '20170501'
    isHoliday = True

    ODFile = open('../Data/metroData_ODFlow.csv','r')
    next(ODFile)

    for row in ODFile: 
        row = row.rstrip().split(',')
        if row[0] != currDate:
            # print(currDate)
            currDate = row[0]
            isHoliday = is_holiday(datetime.datetime.strptime(currDate,'%Y%m%d'))
        
        if isHoliday:
            continue
        
        if int(row[4]) == targetStationID: # evening
            hour = int(row[2])//10000
            if hour >16 and hour <19:
                destinationStationID = int(row[5])
                eveningRushCount[stationID_List.index(destinationStationID)] += int(row[6])#晚上目标站点到各个站点的流量，
        
        if int(row[5]) == targetStationID: # moring 早上各个站点到目标站点的流量
            hour = int(row[2])//10000
            if hour >6 and hour <9:
                originStationID = int(row[4])
                morningRushCount[stationID_List.index(originStationID)] += int(row[6])

    return morningRushCount, eveningRushCount

In [4]:
# 对目标站点, 早高峰 top10 的通勤 Trip 来源； 晚高峰 top10 的通勤 Trip 去向  是否吻合
targetStationID = 247 # 陆家嘴

morningRushCount, eveningRushCount = RushHourConnection(targetStationID, stationID_List)

In [5]:
morningRushCount = np.array(morningRushCount)
eveningRushCount = np.array(eveningRushCount)

indicesMRC = np.argsort(morningRushCount)[::-1]
indicesERC = np.argsort(eveningRushCount)[::-1]

TopTen_MRC_station = [dict_staionName[stationID_List[idx]] for idx in indicesMRC[:10]]
TopTen_ERC_station = [dict_staionName[stationID_List[idx]] for idx in indicesERC[:10]]

print(TopTen_MRC_station)
print(TopTen_ERC_station)


['Xinzha Road', 'Chuangxin Central Road', 'East Xujing', 'South Changjiang Road', 'West Yingao Road', 'Jiuting', 'Caoxi Road', 'Huaqiao', 'Pudong International Airport', 'Fuxing Island']
['Chuangxin Central Road', 'Xinzha Road', 'West Yingao Road', 'East Xujing', 'Pudong International Airport', 'South Changjiang Road', 'Zhongchun Road', 'Jiuting', 'Fuxing Island', 'Sijing']


In [6]:
def displayOnMap(RushCount , TopTen_station_idx, stationID_List, dict_stationLoc, stationInfo, mode):

    map_center = (np.mean(stationInfo["lat"]), np.mean(stationInfo["lon"]))  
    m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB darkmatter')

    # draw all the samples: larger counts -> bigger shape, warmer color 
    colormap = LinearColormap(colors=['blue','white', 'red'],vmin= 0, vmax=np.max(RushCount))

   
    for nodeID, loc in dict_stationLoc.items(): 
        loc = (loc[1], loc[0])
        node_name = dict_staionName[nodeID]
        flowCount = RushCount[stationID_List.index(nodeID)]
        radius = flowCount/np.max(RushCount)*30
        if node_name == '陆家嘴':
            continue
        folium.CircleMarker(location=loc, popup=node_name, radius=radius, color=colormap(flowCount), fillOpacity=0.3, fill=True, stroke=False).add_to(m)

    centerPos = (dict_stationLoc[247][1], dict_stationLoc[247][0])
    folium.Marker(location=centerPos,popup="陆家嘴", icon=folium.Icon(color="yellow", icon="star", prefix='fa')).add_to(m)

    for idx in TopTen_station_idx:
        ID = stationID_List[idx]
        if mode =='moring':
            startPos = (dict_stationLoc[ID][1], dict_stationLoc[ID][0])
            endPos =  centerPos
        else: 
            endPos = (dict_stationLoc[ID][1], dict_stationLoc[ID][0])
            startPos = centerPos

        folium.plugins.AntPath(locations=[startPos, endPos],weight=2,color='green',opacity=0.8, delay=1000).add_to(m)


    return m

In [7]:
displayOnMap(morningRushCount, indicesMRC[:10], stationID_List, dict_stationLoc, stationInfo, 'morning')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28176\3258548236.py:20: UserWarning: color argument of Icon should be one of: {'lightblue', 'darkred', 'darkblue', 'orange', 'green', 'lightgray', 'black', 'blue', 'beige', 'white', 'pink', 'lightred', 'cadetblue', 'gray', 'darkgreen', 'lightgreen', 'red', 'purple', 'darkpurple'}.
  folium.Marker(location=centerPos,popup="陆家嘴", icon=folium.Icon(color="yellow", icon="star", prefix='fa')).add_to(m)


In [8]:
displayOnMap(eveningRushCount, indicesERC[:10], stationID_List, dict_stationLoc, stationInfo, 'evening')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28176\3258548236.py:20: UserWarning: color argument of Icon should be one of: {'lightblue', 'darkred', 'darkblue', 'orange', 'green', 'lightgray', 'black', 'blue', 'beige', 'white', 'pink', 'lightred', 'cadetblue', 'gray', 'darkgreen', 'lightgreen', 'red', 'purple', 'darkpurple'}.
  folium.Marker(location=centerPos,popup="陆家嘴", icon=folium.Icon(color="yellow", icon="star", prefix='fa')).add_to(m)
